# 🍎 AI Fruit Sorter - MobileNet Training
## Train Fresh/Spoiled Classification Model (Miễn Phí với GPU)

### 📋 Hướng Dẫn
1. **Runtime > Change runtime type** → chọn **T4 GPU** (miễn phí)
2. Chạy từng cell theo thứ tự (Shift+Enter)
3. Upload dataset của bạn khi được yêu cầu
4. Chờ training xong (~15-20 phút)
5. Download file `.tflite` về Raspberry Pi

### ⚡ GPU Miễn Phí!
Colab cung cấp GPU miễn phí → Train nhanh hơn 10-20 lần so với CPU!

## 1️⃣ Setup Environment

In [ ]:
# Install dependencies
!pip install -q tensorflow opencv-python matplotlib scikit-learn seaborn

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

print("✅ TensorFlow version:", tf.__version__)
print("✅ GPU available:", "YES" if tf.config.list_physical_devices('GPU') else "NO (using CPU)")

# Create directories
!mkdir -p dataset/train/fresh dataset/train/spoiled
!mkdir -p dataset/val/fresh dataset/val/spoiled
!mkdir -p dataset/test/fresh dataset/test/spoiled
!mkdir -p output

print("\n✅ Setup complete!")

## 2️⃣ Upload Dataset

### Cách 1: Upload File ZIP
Chuẩn bị file ZIP với cấu trúc:
```
dataset.zip
├── train/
│   ├── fresh/
│   │   ├── img1.jpg
│   │   └── ...
│   └── spoiled/
│       └── ...
├── val/
│   ├── fresh/
│   └── spoiled/
└── test/
    ├── fresh/
    └── spoiled/
```

In [ ]:
from google.colab import files
import zipfile

print("📁 Upload file ZIP dataset của bạn...")
uploaded = files.upload()

# Extract ZIP
for filename in uploaded.keys():
    print(f"\n📦 Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('dataset')
    print("✅ Extracted!")

# Count images
def count_images(path):
    return len(list(Path(path).glob('*.[jp][pn][g]*')))

train_fresh = count_images('dataset/train/fresh')
train_spoiled = count_images('dataset/train/spoiled')
val_fresh = count_images('dataset/val/fresh')
val_spoiled = count_images('dataset/val/spoiled')

print(f"\n📊 Dataset Summary:")
print(f"   Train: {train_fresh} fresh, {train_spoiled} spoiled")
print(f"   Val: {val_fresh} fresh, {val_spoiled} spoiled")
print(f"   Total: {train_fresh + train_spoiled + val_fresh + val_spoiled} images")

if train_fresh < 20 or train_spoiled < 20:
    print("\n⚠️ Warning: Ít ảnh! Khuyên dùng ít nhất 50 ảnh mỗi loại")
else:
    print("\n✅ Dataset OK!")

## 3️⃣ Data Augmentation (Optimized)

In [ ]:
from tensorflow.keras import layers

def create_augmentation_pipeline():
    """Enhanced augmentation for conveyor belt conditions"""
    return tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomRotation(0.07),  # 25 degrees
        layers.RandomZoom(0.25),
        layers.RandomTranslation(0.25, 0.25, fill_mode='reflect'),
        layers.RandomBrightness((-0.4, 0.4)),
        layers.RandomContrast(0.3),
        layers.Rescaling(1./127.5, offset=-1)  # Normalize to [-1, 1]
    ], name="augmentation")

def create_data_generator(directory, image_size=(224, 224), batch_size=32, shuffle=True, augment=False):
    """Create data generator"""
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        labels='inferred',
        label_mode='categorical',
        class_names=['fresh', 'spoiled'],
        color_mode='rgb',
        batch_size=batch_size,
        image_size=image_size,
        shuffle=shuffle,
        seed=42
    )
    
    if augment:
        augmentation = create_augmentation_pipeline()
        dataset = dataset.map(
            lambda x, y: (augmentation(x, training=True), y),
            num_parallel_calls=tf.data.AUTOTUNE
        )
    else:
        normalization = layers.Rescaling(1./127.5, offset=-1)
        dataset = dataset.map(
            lambda x, y: (normalization(x), y),
            num_parallel_calls=tf.data.AUTOTUNE
        )
    
    return dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

print("✅ Augmentation pipeline created!")

## 4️⃣ Create Model (With Optimizations)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import models, layers

def create_mobilenet_model(input_shape=(224, 224, 3), num_classes=2, dropout_rate=0.5):
    """Create optimized MobileNetV2 model"""
    
    # Load pretrained base
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze base
    
    # Build model
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dense layers with regularization
    x = layers.Dense(
        256, 
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(
        128, 
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    return model

# Create model
model = create_mobilenet_model()
model.summary()

print("\n✅ Model created!")

## 5️⃣ Prepare Data & Class Weights

In [ ]:
# Load datasets
IMAGE_SIZE = 224
BATCH_SIZE = 32

train_dataset = create_data_generator(
    'dataset/train',
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    augment=True
)

val_dataset = create_data_generator(
    'dataset/val',
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False,
    augment=False
)

# Calculate class weights
total_count = train_fresh + train_spoiled
weight_fresh = total_count / (2 * train_fresh) if train_fresh > 0 else 1.0
weight_spoiled = total_count / (2 * train_spoiled) if train_spoiled > 0 else 1.0
class_weights = {0: weight_fresh, 1: weight_spoiled}

print(f"⚖️ Class weights: Fresh={weight_fresh:.3f}, Spoiled={weight_spoiled:.3f}")
print("✅ Data ready!")

## 6️⃣ Train Model 🚀

In [ ]:
# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        'output/best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Train!
EPOCHS = 50

print("🚀 Starting training...")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Using GPU: {tf.config.list_physical_devices('GPU')}")
print("-" * 60)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

print("\n✅ Training complete!")

## 7️⃣ Evaluate Model

In [ ]:
# Evaluate
results = model.evaluate(val_dataset, verbose=0)

print("📈 Validation Metrics:")
print(f"   Loss: {results[0]:.4f}")
print(f"   Accuracy: {results[1]:.2%}")
print(f"   Precision: {results[2]:.2%}")
print(f"   Recall: {results[3]:.2%}")

# Calculate F1
precision = results[2]
recall = results[3]
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"   F1 Score: {f1:.2%}")

# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Train')
axes[0, 0].plot(history.history['val_accuracy'], label='Val')
axes[0, 0].set_title('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Loss
axes[0, 1].plot(history.history['loss'], label='Train')
axes[0, 1].plot(history.history['val_loss'], label='Val')
axes[0, 1].set_title('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train')
axes[1, 0].plot(history.history['val_precision'], label='Val')
axes[1, 0].set_title('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train')
axes[1, 1].plot(history.history['val_recall'], label='Val')
axes[1, 1].set_title('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('output/training_history.png', dpi=150)
plt.show()

print("\n✅ Evaluation complete!")

## 8️⃣ Convert to TensorFlow Lite

In [ ]:
# Load best model
model = tf.keras.models.load_model('output/best_model.keras')

# Convert to TFLite with optimizations
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]  # Smaller size

tflite_model = converter.convert()

# Save
tflite_path = 'output/mobilenet_classifier.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

# Size info
keras_size = os.path.getsize('output/best_model.keras') / 1024 / 1024
tflite_size = os.path.getsize(tflite_path) / 1024 / 1024

print("📦 Model Conversion:")
print(f"   Keras model: {keras_size:.2f} MB")
print(f"   TFLite model: {tflite_size:.2f} MB")
print(f"   Size reduction: {(1 - tflite_size/keras_size)*100:.1f}%")
print(f"\n✅ TFLite model saved: {tflite_path}")

## 9️⃣ Download Model 📥

In [ ]:
from google.colab import files

print("📥 Downloading model...")
print("\nFiles sẽ tải về:")
print("  1. mobilenet_classifier.tflite ← Copy vào Raspberry Pi")
print("  2. best_model.keras ← Backup")
print("  3. training_history.png ← Biểu đồ training")

files.download('output/mobilenet_classifier.tflite')
files.download('output/best_model.keras')
files.download('output/training_history.png')

print("\n✅ Download complete!")
print("\n📋 Next Steps:")
print("1. Copy mobilenet_classifier.tflite to Raspberry Pi:")
print("   scp mobilenet_classifier.tflite pi@192.168.137.177:~/System_Conveyor/models/")
print("\n2. On Raspberry Pi, run:")
print("   cd ~/System_Conveyor")
print("   python3 fruit_sorter.py")
print("\n🎉 Done!")

---

## ✅ Hoàn Thành!

### 📊 Kết Quả Mong Đợi
- Accuracy: > 90%
- F1 Score: > 90%
- Model size: ~3-4 MB

### 🚀 Deploy to Raspberry Pi
1. Download file `mobilenet_classifier.tflite`
2. Copy to Raspberry Pi:
   ```bash
   scp mobilenet_classifier.tflite pi@192.168.137.177:~/System_Conveyor/models/
   ```
3. Run on Pi:
   ```bash
   cd ~/System_Conveyor
   python3 fruit_sorter.py
   ```

### 💡 Tips
- If accuracy < 85%: Thu thập thêm ảnh
- If overfitting: Tăng augmentation hoặc dropout
- Để train lại: Runtime > Restart and run all

**🎉 Chúc bạn thành công!**